<a href="https://colab.research.google.com/github/mzignis/advance_house_pricing/blob/master/explore_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and settings

In [1]:
HOME = '/content/drive/My Drive/ml_competition/advance_house_pricing'
%cd $HOME

/content/drive/My Drive/ml_competition/advance_house_pricing


In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import RobustScaler, LabelEncoder, OneHotEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
data_dir = os.path.join(HOME, 'data')
os.listdir(data_dir)

['test.csv',
 'sample_submission.csv',
 'data_description.txt',
 'train.csv',
 'preprocessed']

## Inspect data

### Description

In [4]:
with open(os.path.join(data_dir, 'data_description.txt')) as ff:
    description = ff.readlines()

description

['MSSubClass: Identifies the type of dwelling involved in the sale.\t\n',
 '\n',
 '        20\t1-STORY 1946 & NEWER ALL STYLES\n',
 '        30\t1-STORY 1945 & OLDER\n',
 '        40\t1-STORY W/FINISHED ATTIC ALL AGES\n',
 '        45\t1-1/2 STORY - UNFINISHED ALL AGES\n',
 '        50\t1-1/2 STORY FINISHED ALL AGES\n',
 '        60\t2-STORY 1946 & NEWER\n',
 '        70\t2-STORY 1945 & OLDER\n',
 '        75\t2-1/2 STORY ALL AGES\n',
 '        80\tSPLIT OR MULTI-LEVEL\n',
 '        85\tSPLIT FOYER\n',
 '        90\tDUPLEX - ALL STYLES AND AGES\n',
 '       120\t1-STORY PUD (Planned Unit Development) - 1946 & NEWER\n',
 '       150\t1-1/2 STORY PUD - ALL AGES\n',
 '       160\t2-STORY PUD - 1946 & NEWER\n',
 '       180\tPUD - MULTILEVEL - INCL SPLIT LEV/FOYER\n',
 '       190\t2 FAMILY CONVERSION - ALL STYLES AND AGES\n',
 '\n',
 'MSZoning: Identifies the general zoning classification of the sale.\n',
 '\t\t\n',
 '       A\tAgriculture\n',
 '       C\tCommercial\n',
 '       FV\tFloat

### CSV files

In [5]:
train_data_filepath = os.path.join(data_dir, 'train.csv')
train_df_origin = pd.read_csv(train_data_filepath)

train_df_origin.sample()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
996,997,20,RL,NaN,10659,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1961,1961,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,915,Unf,0,135,1050,GasA,...,Y,SBrkr,1050,0,0,1050,1,0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1961.0,Unf,1,368,TA,TA,Y,0,319,0,0,0,0,NaN,NaN,NaN,0,1,2006,COD,Normal,136500


In [6]:
test_data_filepath = os.path.join(data_dir, 'test.csv')
test_df_origin = pd.read_csv(test_data_filepath)

test_df_origin.sample()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1434,2895,120,RM,41.0,5748,Pave,NaN,IR1,HLS,AllPub,Inside,Mod,Crawfor,Norm,Norm,TwnhsE,1Story,8,5,2005,2006,Hip,CompShg,CemntBd,CmentBd,Stone,473.0,Gd,TA,PConc,Ex,TA,Gd,GLQ,1573.0,Unf,0.0,0.0,1573.0,GasA,Ex,Y,SBrkr,1778,0,0,1778,2.0,0.0,2,0,2,1,Ex,5,Typ,1,Gd,Attchd,2005.0,Fin,2.0,495.0,TA,TA,Y,123,53,0,0,153,0,NaN,NaN,NaN,0,2,2006,New,Partial


In [7]:
train_df = train_df_origin.copy()
test_df = test_df_origin.copy()

train_df.drop(columns=['SalePrice'], inplace=True)

In [8]:
train_df.shape, test_df.shape

((1460, 80), (1459, 80))

#### Filling missing values

In [9]:
def get_column_to_remove(df, threshold=20):
    missing_values = df.isna().sum()
    missing_values_df = pd.DataFrame(index=missing_values.index)
    missing_values_df['missing_values'] = missing_values.values
    missing_values_df['missing_values - %'] = missing_values_df['missing_values'] / df.shape[0] * 100

    missing_values_df = missing_values_df[missing_values_df['missing_values'].ne(0)]
    return missing_values_df[missing_values_df['missing_values - %'] > threshold].index.tolist()

In [10]:
get_column_to_remove(train_df)

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [11]:
get_column_to_remove(test_df)

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [12]:
columns_to_drop = get_column_to_remove(train_df)
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

train_df.shape, test_df.shape

((1460, 75), (1459, 75))

In [13]:
train_missing_values_df = train_df[train_df.columns[train_df.isna().sum().astype(bool)].values.tolist()]
train_missing_values_df.head()

,LotFrontage,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond
0,65.0,BrkFace,196.0,Gd,TA,No,GLQ,Unf,SBrkr,Attchd,2003.0,RFn,TA,TA
1,80.0,None,0.0,Gd,TA,Gd,ALQ,Unf,SBrkr,Attchd,1976.0,RFn,TA,TA
2,68.0,BrkFace,162.0,Gd,TA,Mn,GLQ,Unf,SBrkr,Attchd,2001.0,RFn,TA,TA
3,60.0,None,0.0,TA,Gd,No,ALQ,Unf,SBrkr,Detchd,1998.0,Unf,TA,TA
4,84.0,BrkFace,350.0,Gd,TA,Av,GLQ,Unf,SBrkr,Attchd,2000.0,RFn,TA,TA


In [14]:
test_missing_values_df = test_df[test_df.columns[test_df.isna().sum().astype(bool)].values.tolist()]
test_missing_values_df.head()

,MSZoning,LotFrontage,Utilities,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,KitchenQual,Functional,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,SaleType
0,RH,80.0,AllPub,VinylSd,VinylSd,None,0.0,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,0.0,0.0,TA,Typ,Attchd,1961.0,Unf,1.0,730.0,TA,TA,WD
1,RL,81.0,AllPub,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,0.0,0.0,Gd,Typ,Attchd,1958.0,Unf,1.0,312.0,TA,TA,WD
2,RL,74.0,AllPub,VinylSd,VinylSd,None,0.0,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,0.0,0.0,TA,Typ,Attchd,1997.0,Fin,2.0,482.0,TA,TA,WD
3,RL,78.0,AllPub,VinylSd,VinylSd,BrkFace,20.0,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,0.0,0.0,Gd,Typ,Attchd,1998.0,Fin,2.0,470.0,TA,TA,WD
4,RL,43.0,AllPub,HdBoard,HdBoard,None,0.0,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,0.0,0.0,Gd,Typ,Attchd,1992.0,RFn,2.0,506.0,TA,TA,WD


In [15]:
train_missing_values_df.columns[train_missing_values_df.dtypes != object]

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')

In [16]:
test_missing_values_df.columns[test_missing_values_df.dtypes != object]

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')

In [17]:
train_df[train_missing_values_df.columns[train_missing_values_df.dtypes != object]].describe().T

,count,mean,std,min,25%,50%,75%,max
LotFrontage,1201.0,70.049958,24.284752,21.0,59.0,69.0,80.0,313.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.0,0.0,166.0,1600.0
GarageYrBlt,1379.0,1978.506164,24.689725,1900.0,1961.0,1980.0,2002.0,2010.0


In [18]:
test_df[test_missing_values_df.columns[test_missing_values_df.dtypes != object]].describe().T

,count,mean,std,min,25%,50%,75%,max
LotFrontage,1232.0,68.580357,22.376841,21.0,58.00,67.0,80.00,200.0
MasVnrArea,1444.0,100.709141,177.625900,0.0,0.00,0.0,164.00,1290.0
BsmtFinSF1,1458.0,439.203704,455.268042,0.0,0.00,350.5,753.50,4010.0
BsmtFinSF2,1458.0,52.619342,176.753926,0.0,0.00,0.0,0.00,1526.0
BsmtUnfSF,1458.0,554.294925,437.260486,0.0,219.25,460.0,797.75,2140.0
TotalBsmtSF,1458.0,1046.117970,442.898624,0.0,784.00,988.0,1305.00,5095.0
BsmtFullBath,1457.0,0.434454,0.530648,0.0,0.00,0.0,1.00,3.0
BsmtHalfBath,1457.0,0.065202,0.252468,0.0,0.00,0.0,0.00,2.0
GarageYrBlt,1381.0,1977.721217,26.431175,1895.0,1959.00,1979.0,2002.00,2207.0
GarageCars,1458.0,1.766118,0.775945,0.0,1.00,2.0,2.00,5.0


In [19]:
def fill_with_median(df):
    columns = df.columns[df.dtypes != object]
    for column in columns:
        df[column] = df[column].fillna(df[column].median())

In [20]:
fill_with_median(train_df)

In [21]:
fill_with_median(test_df)

In [22]:
def fill_with_mode(df):
    columns = columns = df.columns[df.dtypes == object]
    for column in columns:
        df[column] = df[column].fillna(df[column].mode().iloc[0])

In [23]:
fill_with_mode(train_df)
train_df.columns[train_df.isna().sum().astype(bool)].values.tolist()

[]

In [24]:
fill_with_mode(test_df)
test_df.columns[test_df.isna().sum().astype(bool)].values.tolist()

[]

#### Labeling

In [25]:
text_columns = train_df.columns[train_df.dtypes == object]
number_columns = train_df.columns[train_df.dtypes != object]

In [26]:
def label_data(columns=text_columns):
    for column in columns:
        lbl_encoder = LabelEncoder().fit(train_df[column])
        train_df[f'{column}_encoded'] = lbl_encoder.transform(train_df[column])
        test_df[f'{column}_encoded'] = lbl_encoder.transform(test_df[column])

label_data()

In [27]:
def get_dummies(columns=text_columns):
    train_df_temp = pd.DataFrame()
    test_df_temp = pd.DataFrame()

    for column in text_columns:
        train_data = train_df[column].values.reshape(-1, 1)
        test_data = test_df[column].values.reshape(-1, 1)

        oh_encoder = OneHotEncoder().fit(train_data)
        new_columns = [f'{column}_{x}_oh-encoded' for x in oh_encoder.categories_[0][:-1]]
        new_columns_train_df = pd.DataFrame(oh_encoder.transform(train_data).todense()[:, :-1], columns=new_columns)
        new_columns_test_df = pd.DataFrame(oh_encoder.transform(test_data).todense()[:, :-1], columns=new_columns)

        train_df_temp = pd.concat([train_df_temp, new_columns_train_df], axis=1)
        test_df_temp = pd.concat([test_df_temp, new_columns_test_df], axis=1)

    return train_df_temp, test_df_temp


train_new_columns_df, test_new_columns_df = get_dummies()

In [28]:
train_df = pd.concat([train_df, train_new_columns_df], axis=1)
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,...,Electrical_Mix_oh-encoded,KitchenQual_Ex_oh-encoded,KitchenQual_Fa_oh-encoded,KitchenQual_Gd_oh-encoded,Functional_Maj1_oh-encoded,Functional_Maj2_oh-encoded,Functional_Min1_oh-encoded,Functional_Min2_oh-encoded,Functional_Mod_oh-encoded,Functional_Sev_oh-encoded,GarageType_2Types_oh-encoded,GarageType_Attchd_oh-encoded,GarageType_Basment_oh-encoded,GarageType_BuiltIn_oh-encoded,GarageType_CarPort_oh-encoded,GarageFinish_Fin_oh-encoded,GarageFinish_RFn_oh-encoded,GarageQual_Ex_oh-encoded,GarageQual_Fa_oh-encoded,GarageQual_Gd_oh-encoded,GarageQual_Po_oh-encoded,GarageCond_Ex_oh-encoded,GarageCond_Fa_oh-encoded,GarageCond_Gd_oh-encoded,GarageCond_Po_oh-encoded,PavedDrive_N_oh-encoded,PavedDrive_P_oh-encoded,SaleType_COD_oh-encoded,SaleType_CWD_oh-encoded,SaleType_Con_oh-encoded,SaleType_ConLD_oh-encoded,SaleType_ConLI_oh-encoded,SaleType_ConLw_oh-encoded,SaleType_New_oh-encoded,SaleType_Oth_oh-encoded,SaleCondition_Abnorml_oh-encoded,SaleCondition_AdjLand_oh-encoded,SaleCondition_Alloca_oh-encoded,SaleCondition_Family_oh-encoded,SaleCondition_Normal_oh-encoded
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
test_df = pd.concat([test_df, test_new_columns_df], axis=1)
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,...,Electrical_Mix_oh-encoded,KitchenQual_Ex_oh-encoded,KitchenQual_Fa_oh-encoded,KitchenQual_Gd_oh-encoded,Functional_Maj1_oh-encoded,Functional_Maj2_oh-encoded,Functional_Min1_oh-encoded,Functional_Min2_oh-encoded,Functional_Mod_oh-encoded,Functional_Sev_oh-encoded,GarageType_2Types_oh-encoded,GarageType_Attchd_oh-encoded,GarageType_Basment_oh-encoded,GarageType_BuiltIn_oh-encoded,GarageType_CarPort_oh-encoded,GarageFinish_Fin_oh-encoded,GarageFinish_RFn_oh-encoded,GarageQual_Ex_oh-encoded,GarageQual_Fa_oh-encoded,GarageQual_Gd_oh-encoded,GarageQual_Po_oh-encoded,GarageCond_Ex_oh-encoded,GarageCond_Fa_oh-encoded,GarageCond_Gd_oh-encoded,GarageCond_Po_oh-encoded,PavedDrive_N_oh-encoded,PavedDrive_P_oh-encoded,SaleType_COD_oh-encoded,SaleType_CWD_oh-encoded,SaleType_Con_oh-encoded,SaleType_ConLD_oh-encoded,SaleType_ConLI_oh-encoded,SaleType_ConLw_oh-encoded,SaleType_New_oh-encoded,SaleType_Oth_oh-encoded,SaleCondition_Abnorml_oh-encoded,SaleCondition_AdjLand_oh-encoded,SaleCondition_Alloca_oh-encoded,SaleCondition_Family_oh-encoded,SaleCondition_Normal_oh-encoded
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [30]:
train_df[[x for x in train_df.columns if 'HouseStyle' in x]]

,HouseStyle,HouseStyle_encoded,HouseStyle_1.5Fin_oh-encoded,HouseStyle_1.5Unf_oh-encoded,HouseStyle_1Story_oh-encoded,HouseStyle_2.5Fin_oh-encoded,HouseStyle_2.5Unf_oh-encoded,HouseStyle_2Story_oh-encoded,HouseStyle_SFoyer_oh-encoded
0,2Story,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1Story,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2Story,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,2Story,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2Story,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
1455,2Story,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1456,1Story,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1457,2Story,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1458,1Story,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### Scalling


In [31]:
s = train_df[number_columns].max()
features_to_scalling = s.where(s > 15).dropna().index

def scal_feaures(df, features=features_to_scalling):
    for feature in features:
        scaler = RobustScaler().fit(df[feature].values.reshape(-1, 1))
        df[f'{feature}_scaled'] = scaler.transform(df[feature].values.reshape(-1, 1))

In [32]:
scal_feaures(train_df)
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,...,PavedDrive_N_oh-encoded,PavedDrive_P_oh-encoded,SaleType_COD_oh-encoded,SaleType_CWD_oh-encoded,SaleType_Con_oh-encoded,SaleType_ConLD_oh-encoded,SaleType_ConLI_oh-encoded,SaleType_ConLw_oh-encoded,SaleType_New_oh-encoded,SaleType_Oth_oh-encoded,SaleCondition_Abnorml_oh-encoded,SaleCondition_AdjLand_oh-encoded,SaleCondition_Alloca_oh-encoded,SaleCondition_Family_oh-encoded,SaleCondition_Normal_oh-encoded,Id_scaled,MSSubClass_scaled,LotFrontage_scaled,LotArea_scaled,YearBuilt_scaled,YearRemodAdd_scaled,MasVnrArea_scaled,BsmtFinSF1_scaled,BsmtFinSF2_scaled,BsmtUnfSF_scaled,TotalBsmtSF_scaled,1stFlrSF_scaled,2ndFlrSF_scaled,LowQualFinSF_scaled,GrLivArea_scaled,GarageYrBlt_scaled,GarageArea_scaled,WoodDeckSF_scaled,OpenPorchSF_scaled,EnclosedPorch_scaled,3SsnPorch_scaled,ScreenPorch_scaled,PoolArea_scaled,MiscVal_scaled,YrSold_scaled
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.000000,0.2,-0.210526,-0.254076,0.652174,0.243243,1.193303,0.452790,0.0,-0.559829,-0.269652,-0.453608,1.173077,0.0,0.380070,0.589744,0.281573,0.000000,0.529412,0.0,0.0,0.0,0.0,0.0,0.0
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.998629,-0.6,0.578947,0.030015,0.065217,-0.486486,0.000000,0.834679,0.0,-0.330769,0.538308,0.343643,0.000000,0.0,-0.312090,-0.102564,-0.082816,1.773810,-0.367647,0.0,0.0,0.0,0.0,0.0,-0.5
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.997258,0.2,-0.052632,0.437624,0.608696,0.216216,0.986301,0.143910,0.0,-0.074359,-0.142289,-0.327933,1.189560,0.0,0.497489,0.538462,0.530021,0.000000,0.250000,0.0,0.0,0.0,0.0,0.0,0.0
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.995888,0.4,-0.473684,0.017663,-1.260870,-0.648649,0.000000,-0.235170,0.0,0.106838,-0.468657,-0.247423,1.038462,0.0,0.390885,0.461538,0.670807,0.000000,0.147059,272.0,0.0,0.0,0.0,0.0,-1.0
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.994517,0.2,0.789474,1.181201,0.586957,0.162162,2.130898,0.381186,0.0,0.021368,0.305473,0.113893,1.446429,0.0,1.134029,0.512821,1.474120,1.142857,0.867647,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
scal_feaures(test_df)
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,...,PavedDrive_N_oh-encoded,PavedDrive_P_oh-encoded,SaleType_COD_oh-encoded,SaleType_CWD_oh-encoded,SaleType_Con_oh-encoded,SaleType_ConLD_oh-encoded,SaleType_ConLI_oh-encoded,SaleType_ConLw_oh-encoded,SaleType_New_oh-encoded,SaleType_Oth_oh-encoded,SaleCondition_Abnorml_oh-encoded,SaleCondition_AdjLand_oh-encoded,SaleCondition_Alloca_oh-encoded,SaleCondition_Family_oh-encoded,SaleCondition_Normal_oh-encoded,Id_scaled,MSSubClass_scaled,LotFrontage_scaled,LotArea_scaled,YearBuilt_scaled,YearRemodAdd_scaled,MasVnrArea_scaled,BsmtFinSF1_scaled,BsmtFinSF2_scaled,BsmtUnfSF_scaled,TotalBsmtSF_scaled,1stFlrSF_scaled,2ndFlrSF_scaled,LowQualFinSF_scaled,GrLivArea_scaled,GarageYrBlt_scaled,GarageArea_scaled,WoodDeckSF_scaled,OpenPorchSF_scaled,EnclosedPorch_scaled,3SsnPorch_scaled,ScreenPorch_scaled,PoolArea_scaled,MiscVal_scaled,YrSold_scaled
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.000000,-0.6,0.722222,0.538713,-0.250000,-0.756098,0.000000,0.156250,144.0,-0.328720,-0.203846,-0.359528,0.000000,0.0,-0.888152,-0.444444,0.968992,0.833333,-0.388889,0.0,0.0,120.0,0.0,0.0,1.0
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.998628,-0.6,0.777778,1.179692,-0.312500,-0.829268,0.666667,0.761303,0.0,-0.093426,0.655769,0.491159,0.000000,0.0,-0.170671,-0.518519,-0.651163,2.339286,0.111111,0.0,0.0,0.0,0.0,12500.0,1.0
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.997257,0.2,0.388889,1.073791,0.500000,0.146341,0.000000,0.585771,0.0,-0.558824,-0.115385,-0.296660,1.036982,0.0,0.326429,0.444444,0.007752,1.261905,0.083333,0.0,0.0,0.0,0.0,0.0,1.0
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.995885,0.2,0.611111,0.140313,0.520833,0.146341,0.123457,0.334441,0.0,-0.235294,-0.119231,-0.300589,1.002959,0.0,0.285004,0.469136,-0.038760,2.142857,0.111111,0.0,0.0,0.0,0.0,0.0,1.0
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.994513,1.4,-1.333333,-1.064825,0.395833,0.000000,0.000000,-0.116356,0.0,0.963668,0.561538,0.394892,0.000000,0.0,-0.251864,0.320988,0.100775,0.000000,0.750000,0.0,0.0,144.0,0.0,0.0,1.0


### Saving


In [34]:
train_df.to_csv(os.path.join(data_dir, 'preprocessed', 'train.csv'))
test_df.to_csv(os.path.join(data_dir, 'preprocessed', 'test.csv'))